<a href="https://colab.research.google.com/github/Z3376/Joint-Intent-Classification-and-Slot-Filling/blob/master/dense_joint_intent_classifier3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip /content/gdrive/My\ Drive/Language_2.zip

Archive:  /content/gdrive/My Drive/Language_2.zip
   creating: Language_2/
  inflating: Language_2/.DS_Store    
   creating: __MACOSX/
   creating: __MACOSX/Language_2/
  inflating: __MACOSX/Language_2/._.DS_Store  
  inflating: Language_2/intents.csv  
  inflating: Language_2/Round1_Problem2-of-2_Problem.pdf  
  inflating: __MACOSX/Language_2/._Round1_Problem2-of-2_Problem.pdf  
  inflating: Language_2/slots.csv    
  inflating: Language_2/test.csv     
  inflating: Language_2/train.csv    
  inflating: Language_2/train_intent.csv  
  inflating: Language_2/train_slots.csv  
  inflating: Language_2/vocab.csv    


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import GRU
from keras.layers import LSTM
from keras.models import Model
from keras import backend as K
from keras.layers import Dense
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# Data

In [0]:
train = pd.read_csv('./Language_2/train.csv')
train = train.iloc[:].values
test = pd.read_csv('./Language_2/test.csv')
test = test.iloc[:].values
train_intent = pd.read_csv('./Language_2/train_intent.csv')
train_intent = train_intent.iloc[:].values
intent = pd.read_csv('./Language_2/intents.csv')
intent = intent.iloc[:].values
vocab = pd.read_csv('./Language_2/vocab.csv')
vocab = vocab.iloc[:].values
train_slots = pd.read_csv('./Language_2/train_slots.csv')
train_slots = train_slots.iloc[:].values
slots = pd.read_csv('./Language_2/slots.csv')
slots = slots.iloc[:].values

In [0]:
l = 4000
ltest = len(train)-l

In [0]:
train_data = np.zeros((l,48),dtype='int32')
for i in range(l):
  a = train[i][0].split()
  train_data[i] = np.array([int(a[j])-1 if j<len(a) else 0 for j in range(48)])
train_data = to_categorical(train_data,num_classes=942)

In [0]:
train_out_data = np.zeros((l,48),dtype='int32')
for i in range(l):
  a = train_slots[i][0].split()
  train_out_data[i] = np.array([int(a[j])-1 if j<len(a) else 0 for j in range(48)])
train_out_data = to_categorical(train_out_data,num_classes=128)

In [0]:
train_intent_data = np.array([int(train_intent[i][0]) for i in range(l)])
train_intent_data = to_categorical(train_intent_data)

In [0]:
test_data = np.zeros((ltest,48),dtype='int32')
for i in range(l,len(train)):
  a = train[i-l][0].split()
  test_data[i-l] = np.array([int(a[j])-1 if j<len(a) else 0 for j in range(48)])
test_data = to_categorical(test_data,num_classes=942)

In [0]:
test_out_data = np.zeros((ltest,48),dtype='int32')
for i in range(l,len(train)):
  a = train_slots[i-l][0].split()
  test_out_data[i-l] = np.array([int(a[j])-1 if j<len(a) else 0 for j in range(48)])
test_out_data = to_categorical(test_out_data,num_classes=128)

In [0]:
test_intent_data = np.array([int(train_intent[i-l][0]) for i in range(l,len(train))])
test_intent_data = to_categorical(test_intent_data)

# RNNs

In [0]:
gru_units = 200
max_l = 48

In [0]:
inpt = Input((None,942))
enc_gru = GRU(gru_units,return_sequences=True,return_state=True)
enc_gru_outpt,enc_gru_h = enc_gru(inpt)
slot_outpt = TimeDistributed(Dense(128,activation='softmax'),name='slot_outpt')(enc_gru_outpt)
intent_outpt = Dense(26,activation='softmax',name='intent_outpt')(enc_gru_h)
model = Model(inpt,[slot_outpt,intent_outpt])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 942)    0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, None, 200),  685800      input_1[0][0]                    
__________________________________________________________________________________________________
slot_outpt (TimeDistributed)    (None, None, 128)    25728       gru_1[0][0]                      
__________________________________________________________________________________________________
intent_outpt (Dense)            (None, 26)           5226        gru_1[0][1]                      
Total params: 716,754
Trainable param

In [0]:
plot_model(model, to_file='dense_joint_model.png')

# Train

In [0]:
def seq_metric_fn(s):
  def seq_metric(y_true,y_pred):
    y_mul = np.zeros((s,1))
    for i in range(s):
      y_mul[i] = i
    y_mul = K.cast(y_mul,K.floatx())
    y_true = K.dot(y_true,y_mul)
    y_pred = K.dot(y_pred,y_mul)
    return K.mean(K.square(y_true-y_pred))
  return seq_metric
seq_metric128 = seq_metric_fn(128)
seq_metric26 = seq_metric_fn(26)

In [0]:
output = {'slot_outpt':train_out_data,'intent_outpt':train_intent_data}
losses = {'slot_outpt':'categorical_crossentropy','intent_outpt':'categorical_crossentropy'}
loss_weights = {'slot_outpt':1,'intent_outpt':1}

In [0]:
print(np.argmax(train_data,axis=-1)[5])
print(np.argmax(train_out_data,axis=-1)[5])
print(np.argmax(train_intent_data,axis=-1)[5])

In [0]:
model.compile(optimizer='adam',loss=losses,loss_weights=loss_weights,metrics=['acc'])


In [0]:
#es = EarlyStopping(monitor='val_seq_metric',mode='min',verbose=1,patience=2)
#cp = ModelCheckpoint(filepath='./gdrive/My Drive/Colab Notebooks/slot_weights.hdf5',monitor='val_seq_metric',mode='min', verbose=1,save_weights_only=True, save_best_only=True)

In [0]:
history = model.fit(train_data,output,epochs=10,batch_size=1,validation_split=0.05)

Instructions for updating:
Use tf.cast instead.
Train on 3800 samples, validate on 200 samples
Epoch 1/10
3800/3800 [==============================] - 553s 146ms/step - loss: 1.0778 - slot_outpt_loss: 0.1970 - intent_outpt_loss: 0.8808 - slot_outpt_acc: 0.9632 - intent_outpt_acc: 0.7829 - val_loss: 0.5701 - val_slot_outpt_loss: 0.0533 - val_intent_outpt_loss: 0.5168 - val_slot_outpt_acc: 0.9881 - val_intent_outpt_acc: 0.8850
Epoch 2/10
3800/3800 [==============================] - 448s 118ms/step - loss: 0.4102 - slot_outpt_loss: 0.0471 - intent_outpt_loss: 0.3631 - slot_outpt_acc: 0.9883 - intent_outpt_acc: 0.9071 - val_loss: 0.3170 - val_slot_outpt_loss: 0.0387 - val_intent_outpt_loss: 0.2782 - val_slot_outpt_acc: 0.9905 - val_intent_outpt_acc: 0.9500
Epoch 3/10
3800/3800 [==============================] - 534s 141ms/step - loss: 0.2160 - slot_outpt_loss: 0.0300 - intent_outpt_loss: 0.1860 - slot_outpt_acc: 0.9923 - intent_outpt_acc: 0.9550 - val_loss: 0.2673 - val_slot_outpt_loss: 0.

KeyboardInterrupt: ignored

In [0]:
model.save_weights('dense_joint_model.h5')

In [0]:
print(history.history.keys())
#plt.figure()
#plt.plot(history.history['seq_metric'][1:])
#plt.plot(history.history['val_seq_metric'][1:])
#plt.figure()
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])

NameError: ignored

# Inference

In [0]:
slots_pred = np.zeros((ltest,48,128),dtype='object')
intent_pred = np.zeros((ltest,26),dtype='object')
for i in range(ltest):
  slots_pred[i],intent_pred[i] = model.predict(test_data[i].reshape(1,48,942))

In [0]:
slots_pred_train = np.zeros((l,48,128),dtype='object')
intent_pred_train = np.zeros((l,26),dtype='object')
for i in range(l):
  slots_pred_train[i],intent_pred_train[i] = model.predict(train_data[i].reshape(1,48,942))

In [0]:
n=100
print(np.argmax(slots_pred[n],axis=-1))
a = np.argmax(test_out_data[n],axis=-1)
print(a)

[127 127  43 127 127  47 127  77 127 127  14  11 127   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[127 127  43 127 127  47 127  77 127 127  14  11 127   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [0]:
def accuracy(a,b):
  summ = 0
  for n in range(len(a)):
    an = np.argmax(a[n],axis=-1)
    bn = np.argmax(b[n],axis=-1)
    summ += 1 if np.array_equal(an,bn) else 0
  return float(summ)/len(a)

In [0]:
print('test_accuracy:'+str(accuracy(test_out_data,slots_pred)))
print('train_accuracy:'+str(accuracy(train_out_data,slots_pred_train)))

test_accuracy:0.9416581371545547
train_accuracy:0.936


In [0]:
#print('test_accuracy:'+str(1-np.sum([test_intent_data-np.argmax(intent_pred,axis=-1)])/ltest))
#print('train_accuracy:'+str(1-np.sum([np.argmax(train_intent_data,axis=-1)-np.argmax(intent_pred_train,axis=-1)])/l))

In [0]:
print('test_accuracy:'+str(accuracy(test_intent_data,intent_pred)))
print('train_accuracy:'+str(accuracy(train_intent_data,intent_pred_train)))

test_accuracy:0.9948822927328557
train_accuracy:0.9935


In [0]:
#K.clear_session()